In [1]:
%gui qt
import napari
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from shapely.geometry import Polygon, Point,LineString,  mapping


In [2]:
import json
import geojson
import pathlib


## Make a napari viewer...

In [3]:
v = napari.Viewer()

## Load Cell table from 1 dataset

In [4]:
# downloaded Baysor-segmented data for Baristaseq
cell_stats = pd.read_csv("/Users/brian/Downloads/barista_seq_updated/segmentation_cell_stats.csv")


In [5]:
# add the cells to the napari viewer
v.add_points(cell_stats[["x","y"]].values)

<Points layer 'Points' at 0x12d2d34d0>

In [6]:

# Brian's layer segmentation and restriction of cells to a rectangular region that includes roughly equal area in each layer



with open("/Users/brian/Downloads/baristaseq_annotations_geo.json", 'r') as r:
    layer_geojson = geojson.load(r)
    
layer_data_brl = {layer_annotation["name"]:np.array(layer_annotation["coordinates"][0]) 
                  for layer_annotation in layer_geojson["geometries"] if layer_annotation["name"] not in ["column_axis"]}



In [7]:
# add the polygons to the viewer
v.add_shapes([layer_data_brl[layer_name] for layer_name in layer_data_brl.keys()],shape_type='polygon', edge_width=1,
    edge_color='coral',
    face_color='royalblue',
    name="Layers")

<Shapes layer 'BRL' at 0x10db83d10>

In [8]:
# Use layers to annotate the cell x gene data
brl_layers = v.layers[-1]
is_in_layer = {ii:[Polygon(layer_data_brl[p]).intersects(Point(a))  for a  in cell_stats[["x","y"]].values] for ii,p in enumerate(layer_data_brl.keys())}

In [9]:
annotation_names = ["VISp", "VISp_1", "VISp_2/3", "VISp_4", "VISp_5","VISp_6", "VISp_wm", "column_axis"]


cell_stats["layer"] = "outside_VISp"
for k in is_in_layer.keys():
    cell_stats.loc[is_in_layer[k],["layer"]] = annotation_names[k]


In [10]:
cell_stats

,cell,x,y,has_center,cluster,n_transcripts,density,elongation,area,avg_confidence,doublet_score,layer
0,1,19.142500,179.994429,False,3,70,0.9425,3.624,74.2700,0.9870,0.497700,outside_VISp
1,2,18.728989,240.994511,False,3,47,0.9683,9.837,48.5400,0.9961,0.003106,outside_VISp
2,3,19.650893,251.944857,False,4,28,1.1160,1.274,25.1000,0.9860,NaN,outside_VISp
3,4,21.228078,282.335448,False,2,134,0.9011,2.932,148.7000,0.9964,0.061940,outside_VISp
4,5,21.100798,308.535787,False,3,47,1.0400,3.898,45.2000,0.9991,0.035630,outside_VISp
...,...,...,...,...,...,...,...,...,...,...,...,...
10714,10715,253.218250,592.672500,False,2,12,1.9000,2.278,6.3170,0.9983,NaN,outside_VISp
10715,10716,321.392500,801.450000,False,2,2,NaN,NaN,NaN,0.9986,NaN,VISp_2/3
10716,10717,896.513000,967.688000,False,1,1,NaN,NaN,NaN,0.9709,NaN,VISp_5
10717,10718,501.410000,789.604000,False,2,2,NaN,NaN,NaN,0.9991,NaN,VISp_4


In [11]:
cell_stats.to_csv("/Users/brian/Downloads/barista_seq_updated/segmentation_cell_stats_BRL_layers.csv")